# 21. TF2 API 개요



- Tensorflow V2의 개요와 특징을 파악한다.
- Tensorflow V2의 3가지 주요 API 구성 방식을 이해하고 활용할 수 있다.
- GradientTape를 활용해 보고 좀 더 로우 레벨의 딥러닝 구현 방식을 이해한다.


## 21-2. TensorFlow2 API로 모델 구성하기

간단한 이미지 분류문제를 풀어 tensorflow2를 활용한 모델링을 해보자

방법은 크게 세가지가 있음
- Sequential
- Functional
- Model Subclassing

### 1) TensorFlow2 Sequential Model


In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential()
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)

model.fit(x, y, epochs=10, batch_size=32)  

=> 앞서 공부한 모델의 형식

입력부터 출력까지 레이어를 그야말로 Sequential하게 차곡차곡 add해서 쌓아나가기만 하면 된다.

- 장점 : 초보자가 하기에 매우 쉽다
- 단점 : 모델의 입력과 출력이 여러 개인 경우에는 적합하지 않음(입력 1, 출력1)

### 2) TensorFlow2 Functional API



In [ ]:
import tensorflow as tf
from tensorflow import keras

inputs = keras.Input(shape=(__원하는 입력값 모양__))
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(input)
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)
outputs = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.fit(x,y, epochs=10, batch_size=32)

여기서 다른점은 keras.Model을 쓰는 것이다. -> Sequential Model을 쓰는 것보다 더 일반적인 접근임 왜냐면 시퀀셜은 케라스를 상속받아서 확장한 사례이기 때문이다

- 입력과 출력을 규정함으써 모델 전체를 규정한다.
- input이 되는 텐서가 여러 개가 될 수 있다.
- 그 레이어들을 엮어 output까지 규정한다.
- Functional API를 통해 다중 입력/출력을 가지는 모델을 구성할 수 있음

### 3) TensorFlow2 Subclassing

In [ ]:
import tensorflow as tf
from tensorflow import keras

class CustomModel(keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()
    
    def call(self, x):
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)
        
        return x
    
model = CustomModel()
model.fit(x,y, epochs=10, batch_size=32)

- 제일 자유로운 모델링을 진행할 수 있다.
- 본질적으로는 Functional한 접근이랑 차이가 없음
- keras를 상속받은 모델 클래스를 만드는 것이라서
- init() 메서드 안에서 레이어 구성
- call() 메서드 안에서 레이어 간 forward propagation을 구현
- 레이어에 대한 깊은 이해가 필요

## 21-3. Tensorflow2 API로 모델 작성하기: MNIST (1) Sequential API 활용

Tensorflow2의 다양한 Highlevel API를 활용해서 이미지 문제를 풀어보자

- MNIST 문제 구현

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [5]:
# Sequential Model을 구성해주세요.
"""
Spec:
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [6]:
# 모델 학습 설정

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 9s 3ms/step - loss: 0.1054 - accuracy: 0.9672
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0346 - accuracy: 0.9891
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0200 - accuracy: 0.9937
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0134 - accuracy: 0.9958
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0097 - accuracy: 0.9966
313/313 - 1s - loss: 0.0393 - accuracy: 0.9906


[0.039258308708667755, 0.9905999898910522]

## 21-4. Tensorflow2 API로 모델 작성하기: MNIST (2) Functional API 활용

keras.Input으로 정의된 input 및 output 레이어 구성을 통해 모델 구현

In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [8]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [9]:
"""
Spec:
0. (28X28X1) 차원으로 정의된 Input
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해 주세요.

inputs = keras.Input(shape=(28, 28, 1))

x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
outputs = keras.layers.Dense(10, activation='relu')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [10]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 2.3389 - accuracy: 0.1062
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3024 - accuracy: 0.0988
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3024 - accuracy: 0.0988
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3024 - accuracy: 0.0988
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3023 - accuracy: 0.0989
313/313 - 1s - loss: 2.3026 - accuracy: 0.0980


[2.30259108543396, 0.09799999743700027]

## 21-5. Tensorflow2 API로 모델 작성하기: MNIST (3) Subclassing 활용

- init 메서드 안에서 레이어 선언, call 메서드 안에서 forward propagation 구현
- call 의 입력이 input, 리턴값이 output

In [11]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [ ]:
# Subclassing을 활용한 Model을 구성해주세요.
"""
Spec:
0. keras.Model 을 상속받았으며, __init__()와 call() 메서드를 가진 모델 클래스
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
6. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
"""

class CustomModel(keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')
        self.flat = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128, activation='relu')
        self.fc2 = keras.layers.Dense(10, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flat(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
    
model = CustomModel()

In [12]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 2.3026 - accuracy: 0.0987
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3026 - accuracy: 0.0987
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3026 - accuracy: 0.0987
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3026 - accuracy: 0.0987
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3026 - accuracy: 0.0987
313/313 - 1s - loss: 2.3026 - accuracy: 0.0980


[2.30259108543396, 0.09799999743700027]

## 21-6. TensorFlow2 API로 모델 작성 및 학습하기: CIFAR-100 (1) Sequential API 활용


In [13]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [14]:
# Sequential Model을 구성해주세요.
"""
Spec:
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해주세요
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')
])

In [15]:
# 모델 학습 설정

# 학습 관련 부분을 작성해주세요
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.6352 - accuracy: 0.1544
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9051 - accuracy: 0.2835
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.5981 - accuracy: 0.3476
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.3891 - accuracy: 0.3898
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2182 - accuracy: 0.4252
313/313 - 1s - loss: 2.5675 - accuracy: 0.3602


[2.5674729347229004, 0.3601999878883362]

## 21-7. Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (2) Functional API 활용

keras.Input으로 정의된 input 및 output 레이어 구성을 통해 model 구현

In [16]:
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [17]:
# Functional API를 활용한 Model을 구성해주세요.
"""
Spec:
0. (32X32X3) 차원으로 정의된 Input
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해주세요

inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [18]:
# 모델 학습 설정

# 학습 관련 부분을 작성해주세요
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.5950 - accuracy: 0.1607
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.8906 - accuracy: 0.2854
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.5763 - accuracy: 0.3481
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.3680 - accuracy: 0.3915
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2044 - accuracy: 0.4294
313/313 - 1s - loss: 2.5434 - accuracy: 0.3601


[2.543390989303589, 0.36010000109672546]

## 21-8. Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (3) Subclassing 활용

- init 메서드 안에서 레이어 선언, call 메서드 안에서 forward propagation 구현
- call 의 입력이 input, 리턴값이 output

In [19]:
# Subclassing을 활용한 Model을 구성해주세요.
"""
Spec:
0. keras.Model 을 상속받았으며, __init__()와 call() 메서드를 가진 모델 클래스
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
7. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
"""

class CustomModel(keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flat = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool1(x)
        x = self.maxpool2(x)
        x = self.flat(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
    
model = CustomModel()

In [20]:
# 모델 학습 설정

# 학습 관련 부분을 작성해주세요

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.7301 - accuracy: 0.1347
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.0330 - accuracy: 0.2560
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.7563 - accuracy: 0.3106
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.5773 - accuracy: 0.3465
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4459 - accuracy: 0.3744
313/313 - 1s - loss: 2.6158 - accuracy: 0.3494


[2.615846872329712, 0.34940001368522644]

## 21-9. GradientTape의 활용

### Automatic differentiation - GradientTape

- Tensorflow에서 제공하는 tf.GradientTape는 진행된 모든 연산의 중간 레이어값을 tape에 기록하고, 이를 이용해 gradient를 계산한 후 tape를 폐기하는 기능을 수행한다.
- 이전 스텝에서 진행했던 학습을 tf.GradientTape를 이용한 것으로 변형해보자

In [21]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

50000 10000


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
-> 이부분을 바꿔보자

저렇게 loss, optimizer를 지정해주면 내부적으로는 매 스텝 학습이 진행될 때마다 발생하는 로스와 그래디언트가 model.compile()안에서 자동으로 진행된다.

이제 밑에 코드는 tape.gradient()를 통해 매 스텝 발생하는 그래디언트를 추출한 후 optimizer.apply_gradients()를 통해 발생한 그래디언트가 업데이트 할 파라미터 model.trainable_variavles를 지정해준다

In [24]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

model.fit(x_train, y_train, epochs=5, batch_size=32)
-> 이것도 바꿔보자

배치학습 과정을 위에 train_step()을 호출하도록 바꿔서 구현

In [25]:
import time
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            x_batch.append(x)
            y_batch.append(y)
            if step % batch_size == batch_size-1:
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

Epoch 0: last batch loss = 3.0877
Epoch 1: last batch loss = 2.5430
Epoch 2: last batch loss = 2.2160
Epoch 3: last batch loss = 2.0193
Epoch 4: last batch loss = 1.8471
It took 72.20368504524231 seconds


train_model() = model.fit()과 같음

tf.GradientTape()을 활용해 compile과 fit의 메서드를 꺼내 재구성이 가능하다

강화학습 또는 GAN의 학습을 위해 train_step 메서드의 재구성이 중요하니까 숙지해두자

In [26]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

1/1 [==============================] - 1s 702ms/step


0.3492

충분히 학습을 진행한 상태가 아니므로 최종 accuracy 값은 무시해도 괜찮다! 끝!